# Real estate price prediction

In [20]:
import pandas as pd
pd.set_option("display.max_columns",100)
import numpy as np

from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error

from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("Real estate.csv") #https://www.kaggle.com/datasets/quantbruce/real-estate-price-prediction

### EDA

In [3]:
df

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1
...,...,...,...,...,...,...,...,...
409,410,2013.000,13.7,4082.01500,0,24.94155,121.50381,15.4
410,411,2012.667,5.6,90.45606,9,24.97433,121.54310,50.0
411,412,2013.250,18.8,390.96960,7,24.97923,121.53986,40.6
412,413,2013.000,8.1,104.81010,5,24.96674,121.54067,52.5


In [4]:
df.isnull().sum() #We examine the empty lines.

No                                        0
X1 transaction date                       0
X2 house age                              0
X3 distance to the nearest MRT station    0
X4 number of convenience stores           0
X5 latitude                               0
X6 longitude                              0
Y house price of unit area                0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   No                                      414 non-null    int64  
 1   X1 transaction date                     414 non-null    float64
 2   X2 house age                            414 non-null    float64
 3   X3 distance to the nearest MRT station  414 non-null    float64
 4   X4 number of convenience stores         414 non-null    int64  
 5   X5 latitude                             414 non-null    float64
 6   X6 longitude                            414 non-null    float64
 7   Y house price of unit area              414 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 26.0 KB


In [6]:
df.rename(columns ={'X2 house age':'house_age'},inplace=True)
df.rename(columns ={'X3 distance to the nearest MRT station':'distance_to_the_nearest_MRT_station'},inplace=True)
df.rename(columns ={'X4 number of convenience stores':'number_of_convenience_stores'},inplace=True)
df.rename(columns ={'X5 latitude':'latitude'},inplace=True)
df.rename(columns ={'X6 longitude':'longitude'},inplace=True)
df.rename(columns ={'Y house price of unit area':'house_price_of_unit_area'},inplace=True)
df.rename(columns ={'X1 transaction date':'transaction_date'},inplace=True)
#We change the column names.

In [7]:
df["transaction_date"]=df["transaction_date"].astype("int") #We change the type of data in transaction_date to integer.

In [8]:
df['transaction_date'].value_counts()

2013    288
2012    126
Name: transaction_date, dtype: int64

In [9]:
abs(df.corr()["house_price_of_unit_area"].sort_values(ascending=False)) #We look at their correlations.

house_price_of_unit_area               1.000000
number_of_convenience_stores           0.571005
latitude                               0.546307
longitude                              0.523287
transaction_date                       0.081545
No                                     0.028587
house_age                              0.210567
distance_to_the_nearest_MRT_station    0.673613
Name: house_price_of_unit_area, dtype: float64

### Regression

In [10]:
x,y=df.drop(["No","house_price_of_unit_area","transaction_date"],axis=1),df[["house_price_of_unit_area"]]
x=scaler.fit_transform(x)
x.shape

(414, 5)

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [12]:
model=Sequential()
model.add(Dense(500,activation="relu"))
model.add(Dense(500,activation="relu"))
model.add(Dense(500,activation="relu"))
model.add(Dense(500,activation="relu"))
model.add(Dense(500,activation="relu"))
model.add(Dense(1))
model.compile(optimizer="adam",loss="mse")

In [13]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=10,epochs=500)
model.summary()

Epoch 1/500
34/34 [==============================] - 1s 11ms/step - loss: 544.7482 - val_loss: 133.2844
Epoch 2/500
34/34 [==============================] - 0s 6ms/step - loss: 111.0244 - val_loss: 47.9573
Epoch 3/500
34/34 [==============================] - 0s 5ms/step - loss: 88.0896 - val_loss: 77.4394
Epoch 4/500
34/34 [==============================] - 0s 5ms/step - loss: 84.2519 - val_loss: 43.8768
Epoch 5/500
34/34 [==============================] - 0s 5ms/step - loss: 80.3640 - val_loss: 47.5619
Epoch 6/500
34/34 [==============================] - 0s 5ms/step - loss: 120.0075 - val_loss: 45.2188
Epoch 7/500
34/34 [==============================] - 0s 5ms/step - loss: 76.8272 - val_loss: 41.3398
Epoch 8/500
34/34 [==============================] - 0s 5ms/step - loss: 73.9613 - val_loss: 47.5385
Epoch 9/500
34/34 [==============================] - 0s 5ms/step - loss: 71.8498 - val_loss: 51.8938
Epoch 10/500
34/34 [==============================] - 0s 5ms/step - loss: 75.4348 - va

Epoch 161/500
34/34 [==============================] - 0s 5ms/step - loss: 57.9424 - val_loss: 43.3899
Epoch 162/500
34/34 [==============================] - 0s 5ms/step - loss: 61.5456 - val_loss: 37.0943
Epoch 163/500
34/34 [==============================] - 0s 5ms/step - loss: 56.1122 - val_loss: 40.7493
Epoch 164/500
34/34 [==============================] - 0s 5ms/step - loss: 64.9080 - val_loss: 72.4423
Epoch 165/500
34/34 [==============================] - 0s 5ms/step - loss: 63.4061 - val_loss: 39.3464
Epoch 166/500
34/34 [==============================] - 0s 5ms/step - loss: 56.1137 - val_loss: 37.6010
Epoch 167/500
34/34 [==============================] - 0s 6ms/step - loss: 58.7782 - val_loss: 37.4269
Epoch 168/500
34/34 [==============================] - 0s 5ms/step - loss: 61.2505 - val_loss: 37.1984
Epoch 169/500
34/34 [==============================] - 0s 5ms/step - loss: 58.0940 - val_loss: 40.2520
Epoch 170/500
34/34 [==============================] - 0s 5ms/step - loss

34/34 [==============================] - 0s 5ms/step - loss: 52.5822 - val_loss: 39.1453
Epoch 320/500
34/34 [==============================] - 0s 5ms/step - loss: 50.7500 - val_loss: 78.7284
Epoch 321/500
34/34 [==============================] - 0s 5ms/step - loss: 149.4639 - val_loss: 43.7230
Epoch 322/500
34/34 [==============================] - 0s 5ms/step - loss: 60.1658 - val_loss: 40.4241
Epoch 323/500
34/34 [==============================] - 0s 5ms/step - loss: 56.6326 - val_loss: 43.7043
Epoch 324/500
34/34 [==============================] - 0s 5ms/step - loss: 57.4861 - val_loss: 39.8172
Epoch 325/500
34/34 [==============================] - 0s 5ms/step - loss: 55.7045 - val_loss: 39.6197
Epoch 326/500
34/34 [==============================] - 0s 5ms/step - loss: 51.8174 - val_loss: 45.2746
Epoch 327/500
34/34 [==============================] - 0s 5ms/step - loss: 58.7340 - val_loss: 50.0781
Epoch 328/500
34/34 [==============================] - 0s 5ms/step - loss: 52.2979 - v

34/34 [==============================] - 0s 5ms/step - loss: 47.2801 - val_loss: 34.9015
Epoch 478/500
34/34 [==============================] - 0s 4ms/step - loss: 47.2641 - val_loss: 35.7107
Epoch 479/500
34/34 [==============================] - 0s 5ms/step - loss: 51.2977 - val_loss: 36.4571
Epoch 480/500
34/34 [==============================] - 0s 5ms/step - loss: 65.3626 - val_loss: 35.0513
Epoch 481/500
34/34 [==============================] - 0s 4ms/step - loss: 47.1161 - val_loss: 36.3457
Epoch 482/500
34/34 [==============================] - 0s 5ms/step - loss: 52.2332 - val_loss: 37.3138
Epoch 483/500
34/34 [==============================] - 0s 5ms/step - loss: 48.0937 - val_loss: 33.6841
Epoch 484/500
34/34 [==============================] - 0s 5ms/step - loss: 50.8629 - val_loss: 41.2862
Epoch 485/500
34/34 [==============================] - 0s 5ms/step - loss: 47.2010 - val_loss: 36.3728
Epoch 486/500
34/34 [==============================] - 0s 5ms/step - loss: 46.4079 - va

In [14]:
tahmin=model.predict(x_test)

3/3 [==============================] - 0s 2ms/step


In [15]:
r2_score(tahmin,y_test)

0.7539209920176537

In [16]:
(mean_squared_error(tahmin,y_test))**0.5

5.786938719066521

In [17]:
def algo_test(x,y):
    L = LinearRegression()
    E = ElasticNet()
    R = Ridge()
    Lass = Lasso()
    ETR=ExtraTreeRegressor()
    GBR=GradientBoostingRegressor()
    XGBC= XGBRegressor()
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=13)
    algos = [L,E,R,Lass,ETR,GBR,XGBC]
    algo_names = ['Linear','ElasticNet','Ridge','Lasso','Extra Tree','Gradient Boosting','XGradientBooting']
    r_squared = []
    rmse = []
    mae = []
    result = pd.DataFrame(columns = ['R_Squared','RMSE','MAE'],index = algo_names)
    for algo in algos:
        algo.fit(x_train,y_train)    
        r_squared.append(r2_score(y_test,algo.predict(x_test)))
        rmse.append(mean_squared_error(y_test, algo.predict(x_test))**.5)
        mae.append(mean_absolute_error(y_test, algo.predict(x_test)))
    result.R_Squared = r_squared
    result.RMSE = rmse
    result.MAE= mae
    return result.sort_values('R_Squared', ascending=False)

In [21]:
algo_test(x,y)

,R_Squared,RMSE,MAE
Gradient Boosting,0.709863,6.206374,4.659893
XGradientBooting,0.679097,6.527153,4.584120
Ridge,0.606167,7.230905,5.685921
Linear,0.602466,7.264798,5.667934
Extra Tree,0.499598,8.150727,5.625201
Lasso,0.382042,9.057662,7.005038
ElasticNet,0.148782,10.630585,8.466267
